# Imports

In [1]:
# Import standard libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# API
import requests
import json
import os
# from sodapy import Socrata

# Datetime
import datetime as datetime

import snowflake.connector
# Date
import datetime as dt
import time
from datetime import timedelta

#Math
from math import exp

In [2]:
# # Gets the cursor
# con = snowflake.connector.connect(
#     user='meagan',
#     password='',
#     account='hu40871.us-east-1',
#     warehouse='ANALYSIS'
#     )
# cur = con.cursor()

# # insert query here
# cur.execute('''
#     SELECT  * 
#     FROM    "WAREHOUSE"."GOLD"."SUBSCRIPTION_RETENTION"
#            ''')

## Acuity

In [3]:
silver = pd.read_csv('SILVER_FACT.ACUITY_ND_CONSULT_EVENTS.csv')
silver.head()

FileNotFoundError: [Errno 2] File SILVER_FACT.ACUITY_ND_CONSULT_EVENTS.csv does not exist: 'SILVER_FACT.ACUITY_ND_CONSULT_EVENTS.csv'

In [ ]:
# Create a copy of the DB
silver_acuity = silver.copy()
silver_acuity.shape

In [ ]:
# Change dates to datetime Dtype
silver_acuity['APPOINTMENT_BODY_DATE'] = pd.to_datetime(silver_acuity['APPOINTMENT_BODY_DATE'])
# silver_acuity.columns

# Drop unneeded columns
silver_acuity.drop(columns=['EVENT_TS', 'EVENT_TS_UTC', 'ANONYMOUS_ID','APPOINTMENT_BODY_FIRST_NAME',
       'APPOINTMENT_BODY_DATETIME_CREATED', 'APPOINTMENT_BODY_CALENDAR_ID','APPOINTMENT_BODY_CALENDAR_TIMEZONE',\
                    'APPOINTMENT_BODY_TIMEZONE', 'APPOINTMENT_BODY_LAST_NAME', 'APPOINTMENT_BODY_ID',\
                    'APPOINTMENT_BODY_NO_SHOW', \
                    'APPOINTMENT_TYPE_ID'], inplace=True)

In [ ]:
# Check for null values
sns.heatmap(silver_acuity.isnull(), cbar=False)

In [ ]:
silver_acuity['EVENT'].value_counts()

In [ ]:
silver_acuity['APPOINTMENT_BODY_TYPE'].value_counts()

In [ ]:
# drop irrelevant appointments
na_dict= {'appointment_canceled' : np.nan,
'appointment_changed' : np.nan,
'appointment_rescheduled' : np.nan}

silver_acuity['EVENT'].replace(na_dict, inplace = True)
silver_acuity.dropna(inplace = True)
silver_acuity['EVENT'].value_counts()

In [ ]:
#Round to the day
silver_acuity['APPOINTMENT_BODY_DATE'] = silver_acuity['APPOINTMENT_BODY_DATE'].dt.round('D')

silver_acuity['Today'] = pd.Timestamp('today')
silver_acuity['Today'] = silver_acuity['Today'].dt.round('D')

silver_acuity['Complete'] = (silver_acuity['Today']>silver_acuity['APPOINTMENT_BODY_DATE'])
silver_acuity['Complete'].value_counts()

In [ ]:
# Remove appointments in the future
silver_acuity['Complete'] = silver_acuity['Complete'].astype('str')

# drop irrelevant appointments
na_dict= {'False' : np.nan}

silver_acuity['Complete'].replace(na_dict, inplace = True)
silver_acuity.dropna(inplace = True)
silver_acuity['Complete'].value_counts()

In [ ]:
# Group by event, for a visual only 
chart = silver_acuity.groupby(["APPOINTMENT_BODY_DATE"]).agg({'EVENT':'count'})
chart.isna().sum()

chart.plot(kind='bar',figsize=(20,6))
plt.title("Appointment Date", size= 24)
plt.show()

In [ ]:
(silver_acuity.groupby([silver_acuity['APPOINTMENT_BODY_DATE'].dt.dayofweek,\
                       silver_acuity['APPOINTMENT_BODY_DATE'].dt.month])\
 ['EVENT'].count()).unstack().plot(kind='line')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
min(silver_acuity['APPOINTMENT_BODY_DATE'])

In [ ]:
max(silver_acuity['APPOINTMENT_BODY_DATE'])

In [ ]:
silver_acuity['APPOINTMENT_BODY_DURATION'].value_counts()

In [ ]:
# Looking at specific email addresses
# silver_acuity.loc[silver_acuity['EMAIL']=='XXXX@yahoo.com']

# AGG
clean_acuity = silver_acuity.loc[silver_acuity.groupby('EMAIL')['APPOINTMENT_BODY_DATE'].idxmin()]
clean_acuity.shape

In [ ]:
# yyy = gold.shape
# print ('shape = ',yyy)

smail = clean_acuity['EMAIL'].nunique()
print ('unique emails = ',smail)

## SUB

In [ ]:
gold_sub = pd.read_csv('GOLD.csv')
# gold_sub.head()

In [ ]:
# Create a copy of the dataframe
gold = gold_sub.copy()
gold.columns

In [ ]:
# Change dates to datetime 
gold['FIRST_ORDER_AT'] = pd.to_datetime(gold['FIRST_ORDER_AT'])
gold.info()

In [ ]:
min(gold['FIRST_ORDER_AT'])

In [ ]:
max(gold['FIRST_ORDER_AT'])

In [ ]:
yyy = gold.shape
gemail = gold['EMAIL'].nunique()
print ('shape = ',yyy)
print ('unique emails = ',gemail)

In [ ]:
gold['ITEM_GENDER'].value_counts()

In [ ]:
# clean up work location for 4 main boroughs
gold.loc[gold['ITEM_GENDER'] =='Men,Women', 'ITEM_GENDER']= 'Combination'
gold.loc[gold['ITEM_GENDER'] =='Women,Men,Women', 'ITEM_GENDER']= 'Combination'
gold.loc[gold['ITEM_GENDER'] =='Men,Women,Men,Women,Women,Women,Men,Men', 'ITEM_GENDER']= 'Combination'
gold.loc[gold['ITEM_GENDER'] =='Men,Women,Women', 'ITEM_GENDER']= 'Combination'
gold.loc[gold['ITEM_GENDER'] =='Women,Women,Men', 'ITEM_GENDER']= 'Combination'
gold.loc[gold['ITEM_GENDER'] =='Women,Men,Women,Women', 'ITEM_GENDER']= 'Combination'
gold.loc[gold['ITEM_GENDER'] =='Women,Men', 'ITEM_GENDER']= 'Combination'
gold.loc[gold['ITEM_GENDER'] =='Women,All', 'ITEM_GENDER']= 'Women'
gold.loc[gold['ITEM_GENDER'] =='Women,Women,Women,Women', 'ITEM_GENDER']= 'Women'
gold.loc[gold['ITEM_GENDER'] =='Women,Women', 'ITEM_GENDER']= 'Women'
gold.loc[gold['ITEM_GENDER'] =='Men,Men', 'ITEM_GENDER']= 'Men'


In [ ]:
# clean up data
gold.loc[gold['ITEM_SUB_TYPE'] =='GrowthPlan,GrowthPlan 2', 'ITEM_SUB_TYPE']= 'Combination'
gold.loc[gold['ITEM_SUB_TYPE'] =='Non-Sub,GrowthPlan 3', 'ITEM_SUB_TYPE']= 'Combination'
gold.loc[gold['ITEM_SUB_TYPE'] =='GrowthPlan,Non-Sub', 'ITEM_SUB_TYPE']= 'Combination'
gold.loc[gold['ITEM_SUB_TYPE'] =='GrowthPlan 3,GrowthPlan', 'ITEM_SUB_TYPE']= 'Combination'
gold.loc[gold['ITEM_SUB_TYPE'] =='GrowthPlan,GrowthPlan', 'ITEM_SUB_TYPE']= 'Combination'
gold.loc[gold['ITEM_SUB_TYPE'] =='GrowthPlan 3,GrowthPlan 3', 'ITEM_SUB_TYPE']= 'Combination'
gold.loc[gold['ITEM_SUB_TYPE'] =='GrowthPlan,GrowthPlan 3', 'ITEM_SUB_TYPE']= 'Combination'

gold['ITEM_SUB_TYPE'].value_counts()

In [ ]:
# AGG
df = gold.iloc[gold.groupby('EMAIL')['GROSS_ORDER_VALUE'].idxmin()]
df.shape

In [ ]:
min(df['FIRST_ORDER_AT'])

## Merge

In [ ]:
sub1 = pd.merge(clean_acuity,df,how='left',on='EMAIL')

In [ ]:
gold.columns

In [ ]:
# Create a copy of the dataframe
sub = sub1.copy()

sub = sub[[ 'EMAIL','APPOINTMENT_BODY_DATE', 'FIRST_ORDER_AT',\
           'ITEM_GROUP', 'ITEM_SUB_TYPE','TOTAL_ITEMS','TOTAL_ORDERS',
           'ITEM_NAME', 'ITEM_GENDER', 'GROSS_ORDER_VALUE']]
sub.info()

In [ ]:
max(sub['FIRST_ORDER_AT'])

# # create start year column to be able to make years_working column
# df['start_year'] = df['Agency Start Date'].apply(lambda x: int(x[6:]))
# df['years_with_city'] = df[['Fiscal Year']].sub(df['start_year'], axis=0)

In [ ]:
sub['TimeDifference'] = \
(((sub['APPOINTMENT_BODY_DATE']))-(sub['FIRST_ORDER_AT']))
sub.head()

In [ ]:
sub['ITEM_GROUP'].value_counts()

In [ ]:
sub['ITEM_SUB_TYPE'].value_counts()

In [ ]:
submail = sub['EMAIL'].nunique()
print ('unique emails = ',submail)

In [ ]:
# Export data
sub.to_csv('sub.csv')